# Children in New York Lead levels in their blood 
In this notebook I first read in the data, dropped columns that did not have testable data in them, then split the data into Training and testing sets to check the accuracy of the following question:
    Does the Year of Birth affect whether or not a child would have levels of lead in their system? 

In [2]:
# Dependencies
import numpy as np
import pandas as pd
import datetime as dt
import requests
import json
from pprint import pprint
from matplotlib import style
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report
from imblearn.over_sampling import RandomOverSampler

In [3]:
#  Import and read the combined csv
bt_df=pd.read_csv(Path('../data/cleaned_blood_data.csv'))
bt_df.tail()

County    Zip  Year of Birth  Tests  Less than 5 mcg/dL  5-10 mcg/dL  \
1812  Wyoming  14591           2019      7                 7.0          0.0   
1813    Yates  14527           2017     13                13.0          0.0   
1814    Yates  14527           2020      7                 7.0          0.0   
1815    Yates  14544           2018      8                 8.0          0.0   
1816    Yates  14544           2019      7                 7.0          0.0   

      10-15 mcg/dL  15+ mcg/dL  Total Elevated Blood Levels  Rate per 1,000  \
1812           0.0         0.0                          0.0             0.0   
1813           0.0         0.0                          0.0             0.0   
1814           0.0         0.0                          0.0             0.0   
1815           0.0         0.0                          0.0             0.0   
1816           0.0         0.0                          0.0             0.0   

      Percent        Zip Code Location          County Location  
1812      0.0   (42.827666, -78.08667)  (42.701363, -78.228567)  
1813      0.0  (42.658509, -77.061185)      (42.6431, -77.1485)  
1814      0.0  (42.658509, -77.061185)      (42.6431, -77.1485)  
1815      0.0  (42.742889, -77.245455)      (42.6431, -77.1485)  
1816      0.0  (42.742889, -77.245455)      (42.6431, -77.1485)

In [4]:
#Drop specific columns that are not needed 
bt_df=bt_df.drop(columns=['County','Zip Code Location','County Location'])


In [5]:
#Select your features (columns)
bt_df.columns

Index(['Zip', 'Year of Birth', 'Tests', 'Less than 5 mcg/dL', '5-10 mcg/dL',
       '10-15 mcg/dL', '15+ mcg/dL', 'Total Elevated Blood Levels',
       'Rate per 1,000', 'Percent'],
      dtype='object')

In [6]:
bt_df

Zip  Year of Birth  Tests  Less than 5 mcg/dL  5-10 mcg/dL  \
0     12009           2019     30                30.0          0.0   
1     12023           2019      7                 7.0          0.0   
2     12054           2017      6                 6.0          0.0   
3     12083           2018      6                 6.0          0.0   
4     12083           2019      7                 7.0          0.0   
...     ...            ...    ...                 ...          ...   
1812  14591           2019      7                 7.0          0.0   
1813  14527           2017     13                13.0          0.0   
1814  14527           2020      7                 7.0          0.0   
1815  14544           2018      8                 8.0          0.0   
1816  14544           2019      7                 7.0          0.0   

      10-15 mcg/dL  15+ mcg/dL  Total Elevated Blood Levels  Rate per 1,000  \
0              0.0         0.0                          0.0             0.0   
1              0.0         0.0                          0.0             0.0   
2              0.0         0.0                          0.0             0.0   
3              0.0         0.0                          0.0             0.0   
4              0.0         0.0                          0.0             0.0   
...            ...         ...                          ...             ...   
1812           0.0         0.0                          0.0             0.0   
1813           0.0         0.0                          0.0             0.0   
1814           0.0         0.0                          0.0             0.0   
1815           0.0         0.0                          0.0             0.0   
1816           0.0         0.0                          0.0             0.0   

      Percent  
0         0.0  
1         0.0  
2         0.0  
3         0.0  
4         0.0  
...       ...  
1812      0.0  
1813      0.0  
1814      0.0  
1815      0.0  
1816      0.0  

[1817 rows x 10 columns]

# Split the Data into Training and Testing Sets
Use 'Year of Birth' for the y value to determine if age is a factor or not

In [7]:
#Set features that will be used as our y and x values
y=bt_df['Year of Birth']
X=bt_df.drop(columns='Year of Birth')

In [8]:
#Review the y variable 
y[:5]

0    2019
1    2019
2    2017
3    2018
4    2019
Name: Year of Birth, dtype: int64

In [9]:
#Review the X variable 
X[:5]

Zip  Tests  Less than 5 mcg/dL  5-10 mcg/dL  10-15 mcg/dL  15+ mcg/dL  \
0  12009     30                30.0          0.0           0.0         0.0   
1  12023      7                 7.0          0.0           0.0         0.0   
2  12054      6                 6.0          0.0           0.0         0.0   
3  12083      6                 6.0          0.0           0.0         0.0   
4  12083      7                 7.0          0.0           0.0         0.0   

   Total Elevated Blood Levels  Rate per 1,000  Percent  
0                          0.0             0.0      0.0  
1                          0.0             0.0      0.0  
2                          0.0             0.0      0.0  
3                          0.0             0.0      0.0  
4                          0.0             0.0      0.0

In [10]:
#Check the balance of the targeted value 
y.value_counts()

2018    612
2019    425
2017    306
2016    183
2015    143
2020     98
2014     50
Name: Year of Birth, dtype: int64

In [11]:
#create a Train Test Split 
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state = 42)

## Logistical Regression Model

In [12]:
#Create a Logistic Regression Model by using the training data 
lr_model=LogisticRegression(random_state=42)
#Fit the model using the training Data 
training_model=lr_model.fit(X_train,y_train)

c:\Users\ajain\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [13]:
#make a prediction using testing data 
testing_predictions=training_model.predict(X_test)

In [14]:
#Show the balanced accuracy score of the model
balanced_model=balanced_accuracy_score(y_test, testing_predictions) 
print(balanced_model)

0.14285714285714285


In [15]:
# Generate a confusion matrix for the model
matrix_model=confusion_matrix(y_test,testing_predictions)
print(matrix_model)

[[  0   0   0   0  11   0   0]
 [  0   0   0   0  32   0   0]
 [  0   0   0   0  47   0   0]
 [  0   0   0   0  81   0   0]
 [  0   0   0   0 148   0   0]
 [  0   0   0   0 106   0   0]
 [  0   0   0   0  30   0   0]]


In [16]:
# Print the classification report for the model
classification_model=classification_report(y_test,testing_predictions)
print(classification_model)

              precision    recall  f1-score   support

        2014       0.00      0.00      0.00        11
        2015       0.00      0.00      0.00        32
        2016       0.00      0.00      0.00        47
        2017       0.00      0.00      0.00        81
        2018       0.33      1.00      0.49       148
        2019       0.00      0.00      0.00       106
        2020       0.00      0.00      0.00        30

    accuracy                           0.33       455
   macro avg       0.05      0.14      0.07       455
weighted avg       0.11      0.33      0.16       455



c:\Users\ajain\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\ajain\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\ajain\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [17]:
#Display Results 
print("Confusion Matrix")
display(bt_df)
print(f"Accuracy Score: {balanced_model}")
print("Classification Report")
print(classification_report(y_test,testing_predictions))

Confusion Matrix


Zip  Year of Birth  Tests  Less than 5 mcg/dL  5-10 mcg/dL  \
0     12009           2019     30                30.0          0.0   
1     12023           2019      7                 7.0          0.0   
2     12054           2017      6                 6.0          0.0   
3     12083           2018      6                 6.0          0.0   
4     12083           2019      7                 7.0          0.0   
...     ...            ...    ...                 ...          ...   
1812  14591           2019      7                 7.0          0.0   
1813  14527           2017     13                13.0          0.0   
1814  14527           2020      7                 7.0          0.0   
1815  14544           2018      8                 8.0          0.0   
1816  14544           2019      7                 7.0          0.0   

      10-15 mcg/dL  15+ mcg/dL  Total Elevated Blood Levels  Rate per 1,000  \
0              0.0         0.0                          0.0             0.0   
1              0.0         0.0                          0.0             0.0   
2              0.0         0.0                          0.0             0.0   
3              0.0         0.0                          0.0             0.0   
4              0.0         0.0                          0.0             0.0   
...            ...         ...                          ...             ...   
1812           0.0         0.0                          0.0             0.0   
1813           0.0         0.0                          0.0             0.0   
1814           0.0         0.0                          0.0             0.0   
1815           0.0         0.0                          0.0             0.0   
1816           0.0         0.0                          0.0             0.0   

      Percent  
0         0.0  
1         0.0  
2         0.0  
3         0.0  
4         0.0  
...       ...  
1812      0.0  
1813      0.0  
1814      0.0  
1815      0.0  
1816      0.0  

[1817 rows x 10 columns]

Accuracy Score: 0.14285714285714285
Classification Report
              precision    recall  f1-score   support

        2014       0.00      0.00      0.00        11
        2015       0.00      0.00      0.00        32
        2016       0.00      0.00      0.00        47
        2017       0.00      0.00      0.00        81
        2018       0.33      1.00      0.49       148
        2019       0.00      0.00      0.00       106
        2020       0.00      0.00      0.00        30

    accuracy                           0.33       455
   macro avg       0.05      0.14      0.07       455
weighted avg       0.11      0.33      0.16       455



c:\Users\ajain\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\ajain\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\ajain\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Predicting a Logistical Regression Model with Resampled Training Data 
using RandomOverSampler

In [18]:
# RandomOverSampler to resample the data 
# # Assign a random_state parameter of 42
random_oversampler_model=RandomOverSampler(random_state=42)

# Fit the original training data to the random_oversampler model
X_oversample, y_oversample = random_oversampler_model.fit_resample(X_train,y_train)

In [19]:
# Count the distinct values of the resampled labels data
y_oversample.value_counts()

2014    464
2018    464
2017    464
2016    464
2019    464
2020    464
2015    464
Name: Year of Birth, dtype: int64

In [20]:
#Logistic Regression Model
random_state_model=LogisticRegression(solver='lbfgs', random_state=1)

# Fit the model using the resampled training data
random_state_model.fit(X_oversample,y_oversample)

# Make a prediction using the testing data
y_prediction=random_state_model.predict(X_test)

In [21]:
# Print the balanced_accuracy score of the model 
acc_score=balanced_accuracy_score(y_test, y_prediction)

In [22]:
# Generate a confusion matrix for the model
print(confusion_matrix(y_test, y_prediction))

[[ 0  0  0  2  1  2  6]
 [ 0  0  0  3  7  3 19]
 [ 0  0  0  5 10  3 29]
 [ 0  0  0 11 14  8 48]
 [ 0  0  0 22 49 18 59]
 [ 0  0  0 16 31  5 54]
 [ 0  0  0  4  4  2 20]]


In [23]:
# Print the classification report for the model
print(classification_report(y_test, y_prediction))

              precision    recall  f1-score   support

        2014       0.00      0.00      0.00        11
        2015       0.00      0.00      0.00        32
        2016       0.00      0.00      0.00        47
        2017       0.17      0.14      0.15        81
        2018       0.42      0.33      0.37       148
        2019       0.12      0.05      0.07       106
        2020       0.09      0.67      0.15        30

    accuracy                           0.19       455
   macro avg       0.11      0.17      0.11       455
weighted avg       0.20      0.19      0.17       455



c:\Users\ajain\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\ajain\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\ajain\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [24]:
#Display Results 
print("Confusion Matrix")
display(bt_df)
print(f"Accuracy Score: {acc_score}")
print("Classification Report")
print(classification_report(y_test, y_prediction))


Confusion Matrix


Zip  Year of Birth  Tests  Less than 5 mcg/dL  5-10 mcg/dL  \
0     12009           2019     30                30.0          0.0   
1     12023           2019      7                 7.0          0.0   
2     12054           2017      6                 6.0          0.0   
3     12083           2018      6                 6.0          0.0   
4     12083           2019      7                 7.0          0.0   
...     ...            ...    ...                 ...          ...   
1812  14591           2019      7                 7.0          0.0   
1813  14527           2017     13                13.0          0.0   
1814  14527           2020      7                 7.0          0.0   
1815  14544           2018      8                 8.0          0.0   
1816  14544           2019      7                 7.0          0.0   

      10-15 mcg/dL  15+ mcg/dL  Total Elevated Blood Levels  Rate per 1,000  \
0              0.0         0.0                          0.0             0.0   
1              0.0         0.0                          0.0             0.0   
2              0.0         0.0                          0.0             0.0   
3              0.0         0.0                          0.0             0.0   
4              0.0         0.0                          0.0             0.0   
...            ...         ...                          ...             ...   
1812           0.0         0.0                          0.0             0.0   
1813           0.0         0.0                          0.0             0.0   
1814           0.0         0.0                          0.0             0.0   
1815           0.0         0.0                          0.0             0.0   
1816           0.0         0.0                          0.0             0.0   

      Percent  
0         0.0  
1         0.0  
2         0.0  
3         0.0  
4         0.0  
...       ...  
1812      0.0  
1813      0.0  
1814      0.0  
1815      0.0  
1816      0.0  

[1817 rows x 10 columns]

Accuracy Score: 0.16867428974347212
Classification Report
              precision    recall  f1-score   support

        2014       0.00      0.00      0.00        11
        2015       0.00      0.00      0.00        32
        2016       0.00      0.00      0.00        47
        2017       0.17      0.14      0.15        81
        2018       0.42      0.33      0.37       148
        2019       0.12      0.05      0.07       106
        2020       0.09      0.67      0.15        30

    accuracy                           0.19       455
   macro avg       0.11      0.17      0.11       455
weighted avg       0.20      0.19      0.17       455



c:\Users\ajain\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\ajain\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\ajain\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Schools in New York drinking water safe and does not have lead 
In this notebook I first read in the data, dropped columns that did not have testable data in them, then split the data into Training and testing sets to check the accuracy of the following question:
    Are there any schools in New york that have Lead Free Plumbing? 

In [25]:
#  Import and read the combined csv
df=pd.read_csv(Path('../data/cleaned_water_data.csv'))
df.tail()

County    Zip              School Type of Organization  \
3361  Wyoming  14427       LETCHWORTH HS        Public School   
3362  Wyoming  14427       LETCHWORTH MS        Public School   
3363  Wyoming  14569  WARSAW ELEM SCHOOL        Public School   
3364  Wyoming  14569      WARSAW J S H S        Public School   
3365    Yates  14527    PENN YAN ACADEMY        Public School   

     Any Building with Lead-Free Plumbing?  \
3361                                    No   
3362                                    No   
3363                                    No   
3364                                    No   
3365                 Yes (Entire Building)   

      Number of Outlets that Require Sampling  Outlets ≤ 15 ppb  \
3361                                     49.0              42.0   
3362                                     35.0              28.0   
3363                                     52.0              51.0   
3364                                     47.0              44.0   
3365                                     44.0              43.0   

      Outlets > 15 ppb          County Location  
3361               7.0  (42.723641, -78.204473)  
3362               7.0  (42.723641, -78.204473)  
3363               1.0  (42.723641, -78.204473)  
3364               3.0  (42.723641, -78.204473)  
3365               1.0  (42.634338, -77.078311)

In [26]:
#Drop specific columns that are not needed 
df=df.drop(columns=['County','County Location','Type of Organization','School'])

In [27]:
#Select your features (columns)
df.columns

Index(['Zip', 'Any Building with Lead-Free Plumbing?',
       'Number of Outlets that Require Sampling', 'Outlets ≤ 15 ppb',
       'Outlets > 15 ppb'],
      dtype='object')

In [28]:
df

Zip Any Building with Lead-Free Plumbing?  \
0     12009                                    No   
1     12047                                    No   
2     12047                                    No   
3     12047                                    No   
4     12047                                    No   
...     ...                                   ...   
3361  14427                                    No   
3362  14427                                    No   
3363  14569                                    No   
3364  14569                                    No   
3365  14527                 Yes (Entire Building)   

      Number of Outlets that Require Sampling  Outlets ≤ 15 ppb  \
0                                        63.0              63.0   
1                                        74.0              74.0   
2                                        81.0              80.0   
3                                        53.0              53.0   
4                                        86.0              86.0   
...                                       ...               ...   
3361                                     49.0              42.0   
3362                                     35.0              28.0   
3363                                     52.0              51.0   
3364                                     47.0              44.0   
3365                                     44.0              43.0   

      Outlets > 15 ppb  
0                  0.0  
1                  0.0  
2                  1.0  
3                  0.0  
4                  0.0  
...                ...  
3361               7.0  
3362               7.0  
3363               1.0  
3364               3.0  
3365               1.0  

[3366 rows x 5 columns]

# Split the Data into Training and Testing Sets
Use 'Any Building with Lead-Free Plumbing' for the y value to determine if there are any school building in the New York area with lead free plumbing.

In [29]:
#Set features that will be used as our y and x values
y=df['Any Building with Lead-Free Plumbing?']
X=df.drop(columns='Any Building with Lead-Free Plumbing?')

In [30]:
#Review the y variable 
y[:5]

0    No
1    No
2    No
3    No
4    No
Name: Any Building with Lead-Free Plumbing?, dtype: object

In [31]:
y.value_counts()

No                        3297
Yes (Entire Building)       31
Yes (Partial Building)      22
Yes                         16
Name: Any Building with Lead-Free Plumbing?, dtype: int64

In [32]:
#Review the X variable 
X[:5]

Zip  Number of Outlets that Require Sampling  Outlets ≤ 15 ppb  \
0  12009                                     63.0              63.0   
1  12047                                     74.0              74.0   
2  12047                                     81.0              80.0   
3  12047                                     53.0              53.0   
4  12047                                     86.0              86.0   

   Outlets > 15 ppb  
0               0.0  
1               0.0  
2               1.0  
3               0.0  
4               0.0

In [33]:
#Check the balance of the targeted value 
y.value_counts()

No                        3297
Yes (Entire Building)       31
Yes (Partial Building)      22
Yes                         16
Name: Any Building with Lead-Free Plumbing?, dtype: int64

In [34]:
#create a Train Test Split 
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state = 42)

## Logistical Regression Model

In [35]:
#Create a Logistic Regression Model by using the training data 
lrr_model=LogisticRegression(random_state=42)
#Fit the model using the training Data 
training_model=lrr_model.fit(X_train,y_train)

c:\Users\ajain\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [36]:
#make a prediction using testing data 
testing_predictions=training_model.predict(X_test)

In [37]:
#Show the balanced accuracy score of the model
balanced_model=balanced_accuracy_score(y_test, testing_predictions) 
print(balanced_model)

0.24969843184559712


In [38]:
# Generate a confusion matrix for the model
matrix_model=confusion_matrix(y_test,testing_predictions)
print(matrix_model)

[[828   1   0   0]
 [  3   0   0   0]
 [  9   0   0   0]
 [  1   0   0   0]]


In [39]:
# Print the classification report for the model
classification_model=classification_report(y_test,testing_predictions)
print(classification_model)

                        precision    recall  f1-score   support

                    No       0.98      1.00      0.99       829
                   Yes       0.00      0.00      0.00         3
 Yes (Entire Building)       0.00      0.00      0.00         9
Yes (Partial Building)       0.00      0.00      0.00         1

              accuracy                           0.98       842
             macro avg       0.25      0.25      0.25       842
          weighted avg       0.97      0.98      0.98       842



c:\Users\ajain\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\ajain\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\ajain\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [40]:
#Display Results 
print("Confusion Matrix")
display(df)
print(f"Accuracy Score: {balanced_model}")
print("Classification Report")
print(classification_report(y_test, testing_predictions))

Confusion Matrix


Zip Any Building with Lead-Free Plumbing?  \
0     12009                                    No   
1     12047                                    No   
2     12047                                    No   
3     12047                                    No   
4     12047                                    No   
...     ...                                   ...   
3361  14427                                    No   
3362  14427                                    No   
3363  14569                                    No   
3364  14569                                    No   
3365  14527                 Yes (Entire Building)   

      Number of Outlets that Require Sampling  Outlets ≤ 15 ppb  \
0                                        63.0              63.0   
1                                        74.0              74.0   
2                                        81.0              80.0   
3                                        53.0              53.0   
4                                        86.0              86.0   
...                                       ...               ...   
3361                                     49.0              42.0   
3362                                     35.0              28.0   
3363                                     52.0              51.0   
3364                                     47.0              44.0   
3365                                     44.0              43.0   

      Outlets > 15 ppb  
0                  0.0  
1                  0.0  
2                  1.0  
3                  0.0  
4                  0.0  
...                ...  
3361               7.0  
3362               7.0  
3363               1.0  
3364               3.0  
3365               1.0  

[3366 rows x 5 columns]

Accuracy Score: 0.24969843184559712
Classification Report
                        precision    recall  f1-score   support

                    No       0.98      1.00      0.99       829
                   Yes       0.00      0.00      0.00         3
 Yes (Entire Building)       0.00      0.00      0.00         9
Yes (Partial Building)       0.00      0.00      0.00         1

              accuracy                           0.98       842
             macro avg       0.25      0.25      0.25       842
          weighted avg       0.97      0.98      0.98       842



c:\Users\ajain\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\ajain\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\ajain\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Predicting a Logistical Regression Model with Resampled Training Data 
using RandomOverSampler

In [41]:
# RandomOverSampler to resample the data 
# # Assign a random_state parameter of 42
random_oversampler_model=RandomOverSampler(random_state=42)

# Fit the original training data to the random_oversampler model
X_oversample, y_oversample = random_oversampler_model.fit_resample(X_train,y_train)

In [42]:
# Count the distinct values of the resampled labels data
y_oversample.value_counts()

No                        2468
Yes                       2468
Yes (Partial Building)    2468
Yes (Entire Building)     2468
Name: Any Building with Lead-Free Plumbing?, dtype: int64

In [43]:
#Logistic Regression Model
random_state_model=LogisticRegression(solver='lbfgs', random_state=42)

# Fit the model using the resampled training data
random_state_model.fit(X_oversample,y_oversample)

# Make a prediction using the testing data
y_prediction=random_state_model.predict(X_test)

c:\Users\ajain\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [44]:
# Print the balanced_accuracy score of the model 
acc_score=balanced_accuracy_score(y_test, y_prediction)

In [45]:
# Generate a confusion matrix for the model
print(confusion_matrix(y_test, y_prediction))

[[245 166 321  97]
 [  1   2   0   0]
 [  1   1   5   2]
 [  1   0   0   0]]


In [46]:
# Print the classification report for the model
print(classification_report(y_test, y_prediction))

                        precision    recall  f1-score   support

                    No       0.99      0.30      0.45       829
                   Yes       0.01      0.67      0.02         3
 Yes (Entire Building)       0.02      0.56      0.03         9
Yes (Partial Building)       0.00      0.00      0.00         1

              accuracy                           0.30       842
             macro avg       0.25      0.38      0.13       842
          weighted avg       0.97      0.30      0.45       842



In [47]:
#Display Results 
print("Confusion Matrix")
display(df)
print(f"Accuracy Score: {acc_score}")
print("Classification Report")
print(classification_report(y_test, y_prediction))

Confusion Matrix


Zip Any Building with Lead-Free Plumbing?  \
0     12009                                    No   
1     12047                                    No   
2     12047                                    No   
3     12047                                    No   
4     12047                                    No   
...     ...                                   ...   
3361  14427                                    No   
3362  14427                                    No   
3363  14569                                    No   
3364  14569                                    No   
3365  14527                 Yes (Entire Building)   

      Number of Outlets that Require Sampling  Outlets ≤ 15 ppb  \
0                                        63.0              63.0   
1                                        74.0              74.0   
2                                        81.0              80.0   
3                                        53.0              53.0   
4                                        86.0              86.0   
...                                       ...               ...   
3361                                     49.0              42.0   
3362                                     35.0              28.0   
3363                                     52.0              51.0   
3364                                     47.0              44.0   
3365                                     44.0              43.0   

      Outlets > 15 ppb  
0                  0.0  
1                  0.0  
2                  1.0  
3                  0.0  
4                  0.0  
...                ...  
3361               7.0  
3362               7.0  
3363               1.0  
3364               3.0  
3365               1.0  

[3366 rows x 5 columns]

Accuracy Score: 0.37943975338426483
Classification Report
                        precision    recall  f1-score   support

                    No       0.99      0.30      0.45       829
                   Yes       0.01      0.67      0.02         3
 Yes (Entire Building)       0.02      0.56      0.03         9
Yes (Partial Building)       0.00      0.00      0.00         1

              accuracy                           0.30       842
             macro avg       0.25      0.38      0.13       842
          weighted avg       0.97      0.30      0.45       842

